In [ ]:
# %load_ext autoreload
# %autoreload 2

import os
os.environ["OMP_NUM_THREADS"] ="1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS64_NUM_THREADS"] ="1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] ="1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] ="1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] ="1" # export NUMEXPR_NUM_THREADS=1

'''Set to run on a single core'''


from  astropy.io import fits

import numpy as np
import scipy as sp

import ppxf.sps_util as ppxf_lib
from ppxf_sew import ewfit
from ppxf_sew import ew_util as util
from ppxf import ppxf_util as ppxf_util

from itertools import chain
from functools import partial
from seaborn import kdeplot
from multiprocessing.pool import Pool
import seaborn as sns
import warnings
from matplotlib import colors
warnings.filterwarnings('ignore')
from urllib import request
from ppxf_sew import all_temp_make
from tqdm import trange
from scipy.interpolate import interp1d

import matplotlib as mpl
mpl.rcParams.update (mpl.rcParamsDefault)
mpl.rcParams['lines.linewidth']=2
mpl.rcParams['lines.color']='black'
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['font.size'] = 20
mpl.rcParams['font.family'] = 'DejaVu Serif'
mpl.rcParams['font.style'] = "normal"
mpl.rcParams["mathtext.fontset"] = "stix"
mpl.rcParams['legend.framealpha']=1
mpl.rcParams['xtick.major.width']=2
mpl.rcParams['ytick.major.width']=2
mpl.rcParams['legend.numpoints']=1
mpl.rcParams['xtick.labelsize']=15
mpl.rcParams['ytick.labelsize']=15
mpl.rcParams['legend.fontsize']=20
mpl.rcParams['legend.numpoints']=1
mpl.rcParams['xtick.direction']='in'
mpl.rcParams['ytick.direction']='in'

import matplotlib.ticker as ticker
tl=ticker.MaxNLocator(nbins=5)

from matplotlib.pyplot import MultipleLocator

x_major_locator=MultipleLocator(0.2)
y_major_locator=MultipleLocator(0.2)
import matplotlib.pyplot as plt

c = 299792.458

basic function

In [ ]:

def losvd_rfft(pars, nl, ncomp):
    """

    Analytic Fourier Transform (of real input) of the Gauss-Hermite LOSVD.
    Equation (38) of `Cappellari (2017)
    <https://ui.adsabs.harvard.edu/abs/2017MNRAS.466..798C>`_

    """
    losvd_rfft = np.empty((nl, ncomp), dtype=complex)
    p = 0
    for j in range(ncomp):  # loop over kinematic components
        vel, sig = pars[0 + p], pars[1 + p]
        a = vel/sig
        w = np.linspace(0, np.pi*sig, nl)
        losvd_rfft[:, j] = np.exp(1j*a*w - 0.5*w**2)
        p+=2

    return np.conj(losvd_rfft)

ext = lambda x,delta:(x/5500)**-delta

def att_fun(lam,ebv,delta,f_nodust=0,resp=None):
    # np.random.seed(j)
    # if resp == 'sti':
    #     resp_fun = lambda lam: -0.05*(np.sign(lam-5100)-1)
    # elif resp == 'cal':
    #     resp_fun = lambda lam: -0.05*np.sin(2*np.pi*(lam-5500)/2000)
    # elif resp is None:
    #     resp_fun = lambda lam: 0

    Rv=1/(((89/110)**-delta)-1)

    frac = 10**(-0.4*(ebv*Rv*ext(lam,delta).clip(0)))
    frac = f_nodust + (1-f_nodust)*frac
    if resp is None:
        resp=lambda x: 0
    return frac *10**(-0.4*resp(lam))


power =lambda x: (x/5500)**-1

def mock_file(temp_file,lam_range,weight,comp,params,gas,ebv,ebv1,SN,v_delta, delta=1.3,nn=30 ,savefits=False, AGN=0, resp=None):
    
    hdu=fits.open(os.path.join(os.getcwd(),'test_data/spectrum.fit'))
    header=hdu[0].header
    header['Z']=0
    
    velscale=c* v_delta*np.log(10)

    if temp_file is None:
        sps_name = 'galaxev'

        basename = f"spectra_{sps_name}_9.0.npz"
        temp_file = os.path.join(os.getcwd(),'sps_models/' + basename)

        FWHM_gal=2.76
        sps = ppxf_lib.sps_lib(temp_file, velscale, FWHM_gal, wave_range=[3000,10000])

        temp_star0= sps.templates.reshape(sps.templates.shape[0], -1)
        lam_temp=np.exp(sps.ln_lam_temp)
    else:

        sps=util.temp(temp_file,velscale=velscale,FWHM_gal=2.76,FWHM_tem=2.5,normalize=True)
        temp_star0=sps.templates.reshape(sps.templates.shape[0],-1)

        lam_temp=np.exp(sps.log_lam_temp)

    temp_gas, gas_names, l_wave = util.emission_lines(np.log(lam_temp), 2.76, lam_range_gal=lam_range, comp=comp[gas], params=params, tie_balmer=False, limit_doublets=False)   
 
    if AGN>0:
        temp_star0=np.insert(temp_star0, temp_star0.shape[1], power(lam_temp),  axis=1)
        weight=np.insert(weight, temp_star0.shape[1]-1, AGN,  axis=0)
        weight[:temp_star0.shape[1]]/=np.sum(weight[:temp_star0.shape[1]])
        comp=np.insert(comp, temp_star0.shape[1]-1, True)
        gas=np.insert(gas, temp_star0.shape[1]-1, False)


    npad = 2**int(np.ceil(np.log2(temp_star0.shape[0])))  
    templates_rfft = np.fft.rfft(temp_star0, npad, axis=0)
    lvd_rfft=losvd_rfft(params/velscale,templates_rfft.shape[0],3)
    temp_star=np.array([np.fft.irfft(templates_rfft.T[i]*lvd_rfft[:, comp[~gas][i]], npad)[:lam_temp.shape[0]] for i in range(temp_star0.shape[1])]).T
    temp=np.hstack([temp_star,temp_gas])
    n=temp_star0.shape[1]
    numt=np.where((lam_temp>5400)&(lam_temp<5600))   
    temp[:, :n] /= np.mean(temp[numt].T[:n],1)
    num=(lam_temp>lam_range[0])&(lam_temp<lam_range[1])

    galaxy0=temp@weight

    galaxy_stellar=(temp[:,:nn]@weight[:nn])*att_fun(lam_temp,ebv,delta,resp=resp)*att_fun(lam_temp,ebv1,delta,resp=None)+(temp[:,nn:temp_star0.shape[1]]@weight[nn:temp_star0.shape[1]])*att_fun(lam_temp,ebv,delta,resp=resp)
    
    galaxy_emm=temp[:,temp_star0.shape[1]:]@weight[temp_star0.shape[1]:]

    galaxy=galaxy_stellar/np.mean(galaxy_stellar[numt])
    galaxy+=galaxy_emm
    galaxy[galaxy<=0]=1e-8
    
    noise=galaxy/SN
    galaxy1= np.random.normal(loc=galaxy,scale=noise,size=galaxy.shape[0]) 
    
    lam=lam_temp[num]
    galaxy1=galaxy1[num]
    galaxy=galaxy[num]
    galaxy0=galaxy0[num]
    noise=noise[num]
    galaxy_emm=galaxy_emm[num]
    # temp=temp[num]
    header['COEFF0']=np.log10(lam[0])

    data=[galaxy1,galaxy0,noise]
    if  savefits:
        hdu_img = fits.PrimaryHDU(data=data, header=header)
        hdu_sci = fits.HDUList([hdu_img])
        hdu_sci.writeto(os.path.join(os.getcwd(),'test_data/mock.fits'), overwrite=True,checksum=True)

    return lam, galaxy, galaxy0, galaxy1, galaxy_emm, lam_temp,temp,velscale,noise




def mean_error(matrix, galaxy, noise, weights, grid):
    n, p = matrix.T.shape
    A=(matrix/noise).T
    B=galaxy/noise
    x=weights

    residuals_squared_sum = np.sum(((A@x-B))**2)

    df = n - p

    sigma_squared = residuals_squared_sum / df

    # print(residuals_squared_sum,sigma_squared,n,p)

    num=np.where(x[:np.dot(*grid.shape)]>0)
    # print(num,A.shape)
    inv_ATA = np.linalg.inv(A[:,num[0]].T @ A[:,num[0]])
    cov_matrix = sigma_squared * inv_ATA
    standard_errors = np.sqrt(np.diag(cov_matrix))

    # print(standard_errors)



    p = len(x[num])
    a = grid.reshape(-1)[num]/sum(x[num])
    variance_mean = a.T @ cov_matrix @ a
    se_mean = np.sqrt(variance_mean)*sum(x[num])

    return a@x[num], se_mean


In [ ]:
hdu=fits.open(os.path.join(os.getcwd(),'test_data/spectrum.fit'))
header=hdu[0].header
header['Z']=0
temp_file=os.path.join(os.getcwd(),'sps_models/BC03_SSP_tpl_chab.fits')

lam_range=np.array([3300,9000])
v_delta=1e-4
velscale= c*v_delta*np.log(10)


tie_balmer=False
limit_doublets=False
template=util.temp(temp_file,velscale=velscale,FWHM_gal=2.76,FWHM_tem=2.5,normalize=True)
temp=template.templates.reshape(template.templates.shape[0],-1)
lam_temp=np.exp(template.log_lam_temp)
comp = np.array([0]*120 + [1]*8 + [2]*11)
gas = np.array(comp) >0
params =np.array([50, 80, -10, 100, 30, 50])
gas_templates, gas_names, l_wave = util.emission_lines(np.log(lam_temp), 2.76, lam_range_gal = lam_range, comp=comp[gas], params=params, tie_balmer=tie_balmer, limit_doublets=limit_doublets)   

mock a galaxy

In [ ]:
"""
initial a mock galaxy
"""
n_star=temp.shape[1] 

lam_range=np.array([3300,9000])

wave = np.array([3798.983, 3836.479, 3890.158, 3971.202, 4102.899, 4341.691, 4862.691, 6564.632])  # vacuum wavelengths
wave1=np.array([3726.032, 3728.815, 6716.44 , 6730.816, 3967.467, 3868.763,4685.704, 5875.614, 5006.843, 6300.298, 6583.452])
ratios = np.array([ 0.0530, 0.0731, 0.105, 0.159, 0.259, 0.468, 1, 2.86]) # [0.0530, 0.0731, 0.105, 0.159,]
ratios *= wave[-2]/wave  # Account for varying log-sampled pixel size in Angstrom
num0=np.where((wave>lam_range[0])&(wave<lam_range[1]))
num1=np.where((wave1>lam_range[0])&(wave1<lam_range[1]))

weights=np.zeros(n_star+num0[0].shape[0]+num1[0].shape[0])

weights[n_star:n_star+num0[0].shape[0]]=5*ratios[num0]
weights[n_star+num0[0].shape[0]:]=2*np.ones(num1[0].shape[0])#np.random.random(size=11)
# weights[120:]=0
num=np.arange(0,n_star)
np.random.seed(0)
np.random.shuffle(num)
np.random.seed()
weights[num[:10]]=1./10
comp = np.array([0]*n_star + [1]*8 + [2]*11)
gas = np.array(comp) >0
params =np.array([50, 80, -10, 100, 30, 50])
ebv=0.3
k=0.
ebv1=k*ebv
sn=30
delta=1.3
temp_file=os.path.join(os.getcwd(),'sps_models/BC03_SSP_tpl_chab.fits')
# temp_file=None
AGN=0.
resp=None
lam_range=np.array([3300,9000])
np.random.seed(10)
lam_mock,galaxy1_mock,galaxy0_mock,galaxy1_sn_mock, galaxy_emm_mock,lam_temp0,temp0,velscale,noise=mock_file(temp_file,lam_range,weights,comp,params, gas,ebv,ebv1,sn,v_delta=v_delta,delta=delta,savefits=True,AGN=AGN,resp=resp)
if AGN>0:
    weights=np.insert(weights,120,AGN)
    weights[:121]/=1+AGN
    



In [ ]:
from ppxf_sew import all_temp_make
name='mock.fits'

file=os.path.join(os.getcwd(),'test_data',name)

c = 299792.458

#data read
hdu=fits.open(file)
velscale=c*v_delta*np.log(10)
galaxy=hdu[0].data[0] #galaxy spectrum
galaxy0=hdu[0].data[1] #galaxy spectrum
noise=hdu[0].data[2] #galaxy noise
Z=hdu[0].header['Z'] # galaxy redshift
s=galaxy.shape[0]
mask=np.zeros(s) #galaxy mask
lam=(10**(hdu[0].header['COEFF0']+v_delta*np.arange(s)))/(1+Z) #galaxy lam

# temp read

from ppxf_sew import ew_util as lib


temp_file=os.path.join(os.getcwd(),'sps_models/BC03_SSP_tpl_chab.fits')
template=lib.temp(temp_file,velscale=velscale,FWHM_gal=2.76,FWHM_tem=2.5,normalize=True) #stellar template

temp=template.templates.reshape(template.templates.shape[0],-1)
lam_temp=np.exp(template.log_lam_temp)
lam_range_gal = np.array([np.min(lam), np.max(lam)])

AGN=0.

n_star=temp.shape[1] 


templates,start,moments,component,gas_component,gas_names,gas_wave=all_temp_make(temp,lam_temp,lam_range_gal)
if AGN>0:
    templates=np.insert(templates,120,power(lam_temp),axis=1)
# np.hstack([templates,np.array([power(lam_temp)]).T])
vlim = np.array([-2900, 2900])  # As 2e3 nonlinear_fit() +900 for 3sigma
lam_range = lam_temp[[0, -1]]/np.exp(vlim/c)
num=(lam > lam_range[0]) & (lam < lam_range[1])
lam=lam[num]
galaxy=galaxy[num]
# galaxy1=galaxy1[num]
# galaxy_emm=galaxy_emm[num]
galaxy1=galaxy1_mock[num]
galaxy0=galaxy0_mock[num]
noise=noise[num]
mask=mask[num]


In [ ]:
"""
input weights and attenuated galaxy spectrum, initial galaxy spectrum, attenuation curve
"""
# sps_ppxf = ppxf_lib.sps_lib(temp_file, velscale, FWHM_gal, wave_range=[3000,10000])
temp_file=os.path.join(os.getcwd(),'sps_models/BC03_SSP_tpl_chab.fits')
sps_bc03 = util.temp(temp_file,velscale=velscale,FWHM_gal=2.76,FWHM_tem=2.5,normalize=True)
plt.figure(figsize=(20,5))
ax1=plt.axes([0.05,0.1,0.3,0.8])
n_star=120
n_age=sps_bc03.templates.shape[1]
n_metal=sps_bc03.templates.shape[2]
sps_bc03.plot(weights[:n_star].reshape(n_age,n_metal))
ax2=plt.axes([0.38,0.1,0.6,0.8])
plt.plot(lam,galaxy,'k',label='input att galaxy')
# plt.plot(lam,galaxy0,'r',label='input att galaxy')
# plt.plot(lam_mock,galaxy1_mock,label='input init galaxy')
# plt.plot(lam,galaxy,'--',label='input att galaxy')
plt.plot(lam_mock,galaxy0_mock,label='input init galaxy')
plt.plot(lam,galaxy1,label='input init galaxy')
# plt.plot(lam,galaxy0*att_fun(lam,ebv,1.3)*2.6)
# plt.plot(lam_temp,(templates[:,comp==0]@weights[comp==0]*att_fun(lam_temp,ebv,1.3))+(templates[:,comp!=0]@weights[comp!=0]),'--',label='att galaxy')
# plt.plot(lam_temp0,(temp0[:,comp==0]@weights[comp==0]*att_fun(lam_temp0,ebv,1.3))+(temp0[:,comp!=0]@weights[comp!=0]),'r--',label='att galaxy')
plt.xlabel(r'$\lambda(\AA)$',fontsize=10)
# plt.xlim(3600,4500)
plt.legend(fontsize=15)


In [ ]:
from ppxf.ppxf import ppxf as pp
ext = lambda x,delta:(x/5500)**-delta


def att_fun1(lam,ebv,delta,f_nodust=0):
    # delta=-np.log(1+1/Rv)/np.log(89/110)
    
    Rv=1/(((89/110)**-delta)-1)
    # k=(110.-89.**2/110./Rv)/21.
    # ext = lambda x:k*(x/5500)**-1+(1-k)*(x/5500)**-2
    frac = 10**(-0.4*ebv*Rv*ext(lam,delta).clip(0))
    frac = f_nodust + (1-f_nodust)*frac
    return frac

start = [0, 180.]
# params =np.array([50, 80, -10, 100, 30, 50])
n_temps = temp.shape[1] 
n_forbidden = np.sum(["[" in a or "HeI" in a for a in gas_names])
n_balmer = len(gas_names) - n_forbidden
component = np.array([0]*n_temps + [1]*n_balmer + [2]*n_forbidden)
if AGN>0:
    component = np.insert(component,120,0)
gas_component = np.array(component) >0
moments = [2,2,2]
start = [start, start[:2], start[:2]]


dust2= [{"start": [0.5,1.3], "bounds": [[0, 2], [0.5,1.5]], "func":att_fun1, "component": ~gas_component,"fixed":[False,False]}]
dust3= [{"start": [0.5], "bounds": [[0, 2]], "component": ~gas_component,"fixed":[False]}]



In [ ]:
'''
SEW
'''

%time p1 = ewfit(templates, galaxy, noise, velscale, start,Z=Z, moments=moments ,degree=-1,dust=None,linear_method='lsq_box',plot=False,clean=False, lam=lam, lam_temp=lam_temp, component=component,gas_component=gas_component, gas_names=gas_names, tie_balmer= tie_balmer, limit_doublets=limit_doublets,l=1e-8)

In [ ]:
p1.plot()

In [ ]:
'''
ppxf+power attenuation curve
'''
%time p2=pp(templates, galaxy, noise, velscale, start, moments=moments, degree=-1, dust=dust2, linear_method='lsq_box', plot=False, clean=False, lam=lam, lam_temp=lam_temp, component=component, gas_component=gas_component, gas_names=gas_names,quiet=False)

In [ ]:
p2.plot()

In [ ]:
'''
ppxf+calzetti curve
'''
p3=pp(templates, galaxy, noise, velscale, start, moments=moments, degree=-1 ,dust=dust3,linear_method='lsq_box',plot=False,clean=False,lam=lam, lam_temp=lam_temp, component=component,gas_component=gas_component, gas_names=gas_names)

In [ ]:
sps_bc03.mean_age_metal(weights[:n_star].reshape(n_age,n_metal))
sps_bc03.mean_age_metal(p1.weights[:n_temps].reshape(n_age,n_metal))
print(mean_error(p1.tmp, p1.galaxy, p1.noise, p1.weights, template.age_grid), mean_error(p1.tmp, p1.galaxy, p1.noise, p1.weights, template.metal_grid))
sps_bc03.mean_age_metal(p2.weights[:n_temps].reshape(n_age,n_metal))
print(mean_error(p2.matrix.T, p2.galaxy, p2.noise, p2.weights, template.age_grid), mean_error(p2.matrix.T, p2.galaxy, p2.noise, p2.weights, template.metal_grid))
sps_bc03.mean_age_metal(p3.weights[:n_temps].reshape(n_age,n_metal))
print(mean_error(p3.matrix.T, p3.galaxy, p3.noise, p3.weights, template.age_grid), mean_error(p3.matrix.T, p3.galaxy, p3.noise, p3.weights, template.metal_grid))


In [ ]:
plt.figure(figsize=(20,20))
plt.axes([0.05,0.78,0.23,0.2])
n_age=sps_bc03.templates.shape[1]
n_metal=sps_bc03.templates.shape[2]
sps_bc03.plot(weights[:n_temps].reshape(n_age,n_metal),title='mock weights',ylabel=r'$[Z/Z_\odot]$',vmin=0,vmax=0.25)


plt.axes([0.05,0.53,0.23,0.2])
sps_bc03.plot(p1.weights[:n_temps].reshape(n_age,n_metal),title='weights of SEW',ylabel=r'$[Z/Z_\odot]$',vmin=0,vmax=0.25)
plt.axes([0.05,0.28,0.23,0.2])
sps_bc03.plot(p2.weights[:n_temps].reshape(n_age,n_metal)/np.sum(p2.weights[:n_temps]),title='weights of pPXF+mock',ylabel=r'$[Z/Z_\odot]$',vmin=0,vmax=0.25)
plt.axes([0.05,0.03,0.23,0.2])
sps_bc03.plot(p3.weights[:n_temps].reshape(n_age,n_metal)/np.sum(p3.weights[:n_temps]),title='weights of pPXF+misfit',ylabel=r'$[Z/Z_\odot]$',vmin=0,vmax=0.25)

plt.axes([0.325,0.78,0.66,0.2])
plt.title('final mock spectrum')
plt.plot(lam_mock,galaxy0_mock,'k',label = 'mock intrinsic spectrum')
plt.plot(p1.lam,p1.galaxy,'grey', label = 'final mock spectrum')
plt.plot(p1.lam,p1.galaxy_l, 'dodgerblue', label = 'smoothed spectrum')
plt.xlabel(r'$\lambda(\AA)$')
plt.ylabel('normed flux',fontsize=20)
plt.xlim(3500,9000)
plt.legend(fontsize=15)

plt.axes([0.325,0.53,0.66,0.2])
plt.title('EW process and fit')
plt.plot(p1.lam,p1.galaxy/p1.galaxy_l,'grey',label='EW-processed spectrum')
plt.plot(p1.lam,(p1.gal_temp[:,p1.degree+1:][:,~p1.gas_component]@weights[~p1.gas_component])/(p1.gal_temp_l[:,p1.degree+1:][:,~p1.gas_component]@weights[~p1.gas_component])+(p1.gal_temp[:,p1.degree+1:][:,p1.gas_component]@weights[p1.gas_component])/p1.galaxy_l,'k',label='intrinsic EW spectrum')
plt.plot(p1.lam,(p1.gal_temp[:,p1.degree+1:][:,~p1.gas_component]@weights[~p1.gas_component])/(p1.gal_temp_l[:,p1.degree+1:][:,~p1.gas_component]@weights[~p1.gas_component]),'k--')
plt.plot(p1.lam,(p1.gal_temp[:,p1.degree+1:][:,~gas_component]@p1.weights[~gas_component])/(p1.gal_temp_l[:,p1.degree+1:][:,~gas_component]@p1.weights[~gas_component])+(p1.gal_temp[:,p1.degree+1:][:,gas_component]@p1.weights[gas_component])/p1.galaxy_l,'dodgerblue',ls='--',label='fit EW spectrum')
plt.plot(p1.lam,(p1.gal_temp[:,p1.degree+1:][:,~gas_component]@p1.weights[~gas_component])/(p1.gal_temp_l[:,p1.degree+1:][:,~gas_component]@p1.weights[~gas_component]),'dodgerblue',ls='-.')
plt.legend(fontsize=15)
plt.plot([3700,3750],[1.5,1.67],'k')
plt.plot([4150,6500],[1.5,1.67],'k')
plt.plot([3700,4150,4150,3700,3700],[0.5,0.5,1.5,1.5,0.5],'k')
plt.xlim(3500,9000)
plt.xlabel(r'$\lambda(\AA)$')
plt.ylabel('EW',fontsize=20)

ax=plt.axes([0.355,0.615,0.33,0.11])
ax.patch.set_alpha(0)
plt.plot(p1.lam,p1.galaxy/p1.galaxy_l,'grey',label='EW spectrum')
plt.plot(p1.lam,(p1.gal_temp[:,p1.degree+1:][:,~p1.gas_component]@weights[~p1.gas_component])/(p1.gal_temp_l[:,p1.degree+1:][:,~p1.gas_component]@weights[~p1.gas_component])+(p1.gal_temp[:,p1.degree+1:][:,p1.gas_component]@weights[p1.gas_component])/p1.galaxy_l,'k',label='intrinsic EW spectrum')
plt.plot(p1.lam,(p1.gal_temp[:,p1.degree+1:][:,~p1.gas_component]@weights[~p1.gas_component])/(p1.gal_temp_l[:,p1.degree+1:][:,~p1.gas_component]@weights[~p1.gas_component]),'k--')
plt.plot(p1.lam,(p1.gal_temp[:,p1.degree+1:][:,~gas_component]@p1.weights[~gas_component])/(p1.gal_temp_l[:,p1.degree+1:][:,~gas_component]@p1.weights[~gas_component])+(p1.gal_temp[:,p1.degree+1:][:,gas_component]@p1.weights[gas_component])/p1.galaxy_l,'dodgerblue',ls='--',label='best-fit EW spectrum')
plt.plot(p1.lam,(p1.gal_temp[:,p1.degree+1:][:,~gas_component]@p1.weights[~gas_component])/(p1.gal_temp_l[:,p1.degree+1:][:,~gas_component]@p1.weights[~gas_component]),'dodgerblue',ls='-.')
plt.xlim(3700,4150)
plt.ylim(0.5,1.5)

plt.axes([0.325,0.28,0.66,0.2])
plt.title('full spectrum')
plt.plot(p1.lam,p1.galaxy,'grey',lw=3, label = 'final mock spectrum')
plt.plot(lam_mock,galaxy0_mock,'k',lw=3, label = 'mock intrinsic spectrum')
plt.plot(p1.lam,p1.gal_temp[:,p1.degree+1:]@p1.weights,'dodgerblue',ls='--',lw=2, dashes=(10,5),label='SEW spectrum')
plt.plot(p1.lam,p1.gal_temp[:,p1.degree+1:][:,~gas_component]@p2.weights[~gas_component]/np.sum(p2.weights[~gas_component])+p1.gal_temp[:,p1.degree+1:][:,gas_component]@p2.weights[gas_component],'r--',lw=2, dashes=(3,5),label='pPXF+mock')
plt.plot(p1.lam,p1.gal_temp[:,p1.degree+1:][:,~gas_component]@p3.weights[~gas_component]/np.sum(p3.weights[~gas_component])+p1.gal_temp[:,p1.degree+1:][:,gas_component]@p2.weights[gas_component],'m--',lw=2,label='pPXF+misfit')
plt.legend(fontsize=15)
plt.plot([3700,3750],[2,2.2],'k')
plt.plot([4150,6500],[2,2.2],'k')
plt.plot([3700,4150,4150,3700,3700],[0.5,0.5,2,2,0.5],'k')
plt.xlim(3500,9000)
plt.ylim(0.3,4.78)
plt.xlabel(r'$\lambda(\AA)$')
plt.ylabel('normed flux',fontsize=20)

ax=plt.axes([0.355,0.365,0.33,0.11])
ax.patch.set_alpha(0)
plt.plot(p1.lam,p1.galaxy,'grey',lw=5, label = 'final mock spectrum')
plt.plot(lam_mock,galaxy0_mock,'k',lw=5, label = 'mock intrinsic spectrum')
plt.plot(p1.lam,p1.gal_temp[:,p1.degree+1:]@p1.weights,'dodgerblue',ls='--',lw=3, dashes=(10,5),label='SEW spectrum')


plt.plot(p1.lam,p1.gal_temp[:,p1.degree+1:][:,~gas_component]@p2.weights[~gas_component]/np.sum(p2.weights[~gas_component])+p1.gal_temp[:,p1.degree+1:][:,gas_component]@p2.weights[gas_component],'r--',lw=3, dashes=(3,5),label='pPXF+mock')
plt.plot(p1.lam,p1.gal_temp[:,p1.degree+1:][:,~gas_component]@p3.weights[~gas_component]/np.sum(p3.weights[~gas_component])+p1.gal_temp[:,p1.degree+1:][:,gas_component]@p2.weights[gas_component],'m--',lw=3,label='pPXF+misfit')
plt.xlim(3700,4150)
plt.ylim(0.5,2)


plt.axes([0.325,0.03,0.66,0.2])
plt.title('attenuation curve')
plt.plot(p1.lam, p1.att_curve,'b.',label='SEW',alpha=0.3)
ebv=0.3
ext = lambda x,delta:(x/5500)**-delta
att = lambda x, ebv, delta: ebv*(ext(x,delta)-1)/(((89/110)**-delta)-1)

plt.plot(p1.lam,att(p1.lam,ebv,1.3),'k--',lw=5,label='mock attenuaiton')


plt.plot(p1.lam, p1.att_curve_smooth,'dodgerblue',ls='--',lw=3,label='smoothed SEW')

plt.plot(p2.lam,-2.5*np.log10(p2.dust[0]['func'](p2.lam,*p2.dust[0]['sol'])/p2.dust[0]['func'](5500,*p2.dust[0]['sol'])),'r--',lw=3,label='pPXF+mock')

plt.plot(p3.lam,-2.5*np.log10(p3.dust[0]['func'](p3.lam,*p3.dust[0]['sol'])/p3.dust[0]['func'](5500,*p3.dust[0]['sol'])),'m--',lw=3,label='pPXF+misfit')
plt.xlabel(r'$\lambda(\AA)$')
plt.ylabel(r'$A_\lambda-A_V$',fontsize=20)
plt.xlim(3550,9000)
plt.ylim(-0.7,1.)
plt.legend(ncol=2,fontsize=15)

plt.tight_layout()

mock calibration bias

In [ ]:
"""
initial a mock galaxy
"""
n_star=temp.shape[1] 

lam_range=np.array([3300,9000])

wave = np.array([3798.983, 3836.479, 3890.158, 3971.202, 4102.899, 4341.691, 4862.691, 6564.632])  # vacuum wavelengths
wave1=np.array([3726.032, 3728.815, 6716.44 , 6730.816, 3967.467, 3868.763,4685.704, 5875.614, 5006.843, 6300.298, 6583.452])
ratios = np.array([ 0.0530, 0.0731, 0.105, 0.159, 0.259, 0.468, 1, 2.86]) # [0.0530, 0.0731, 0.105, 0.159,]
ratios *= wave[-2]/wave  # Account for varying log-sampled pixel size in Angstrom
num0=np.where((wave>lam_range[0])&(wave<lam_range[1]))
num1=np.where((wave1>lam_range[0])&(wave1<lam_range[1]))

weights=np.zeros(n_star+num0[0].shape[0]+num1[0].shape[0])

weights[n_star:n_star+num0[0].shape[0]]=5*ratios[num0]
weights[n_star+num0[0].shape[0]:]=2*np.ones(num1[0].shape[0])#np.random.random(size=11)
# weights[120:]=0
num=np.arange(0,n_star)
np.random.seed(0)
np.random.shuffle(num)
np.random.seed()
weights[num[:10]]=1./10
comp = np.array([0]*n_star + [1]*8 + [2]*11)
gas = np.array(comp) >0
params =np.array([50, 80, -10, 100, 30, 50])
ebv=0.3
k=0.
ebv1=k*ebv
sn=30
delta=1.3
temp_file=os.path.join(os.getcwd(),'sps_models/BC03_SSP_tpl_chab.fits')
AGN=0.
resp_type='cal'

T=2500
del_A=0.05
if resp_type == 'sti':
    resp = lambda lam: -0.05*(np.sign(lam-5100)-1)
elif resp_type == 'cal':
    resp = lambda lam: -del_A*np.sin(2*np.pi*(lam-5500)/T)
elif resp_type is None:
    resp = lambda lam: 0*lam
lam_range=np.array([3300,9000])
np.random.seed(10)
lam_mock,galaxy1_mock,galaxy0_mock,galaxy1_sn_mock, galaxy_emm_mock,lam_temp0,temp0,velscale,noise=mock_file(temp_file,lam_range,weights,comp,params, gas,ebv,ebv1,sn,v_delta=v_delta,delta=delta,savefits=True,AGN=AGN,resp=resp)
if AGN>0:
    weights=np.insert(weights,120,AGN)
    weights[:121]/=1+AGN
    



In [ ]:
from ppxf_sew import all_temp_make
name='mock.fits'

file=os.path.join(os.getcwd(),'test_data',name)

c = 299792.458

#data read
hdu=fits.open(file)
velscale=c*v_delta*np.log(10)
galaxy=hdu[0].data[0] #galaxy spectrum
galaxy0=hdu[0].data[1] #galaxy spectrum
noise=hdu[0].data[2] #galaxy noise
Z=hdu[0].header['Z'] # galaxy redshift
s=galaxy.shape[0]
mask=np.zeros(s) #galaxy mask
lam=(10**(hdu[0].header['COEFF0']+v_delta*np.arange(s)))/(1+Z) #galaxy lam

# temp read

from ppxf_sew import ew_util as lib


temp_file=os.path.join(os.getcwd(),'sps_models/BC03_SSP_tpl_chab.fits')
template=lib.temp(temp_file,velscale=velscale,FWHM_gal=2.76,FWHM_tem=2.5,normalize=True) #stellar template

temp=template.templates.reshape(template.templates.shape[0],-1)
lam_temp=np.exp(template.log_lam_temp)
lam_range_gal = np.array([np.min(lam), np.max(lam)])

AGN=0.

n_star=temp.shape[1] 


templates,start,moments,component,gas_component,gas_names,gas_wave=all_temp_make(temp,lam_temp,lam_range_gal)
if AGN>0:
    templates=np.insert(templates,120,power(lam_temp),axis=1)
vlim = np.array([-2900, 2900]) 
lam_range = lam_temp[[0, -1]]/np.exp(vlim/c)
num=(lam > lam_range[0]) & (lam < lam_range[1])
lam=lam[num]
galaxy=galaxy[num]

galaxy1=galaxy1_mock[num]
galaxy0=galaxy0_mock[num]
noise=noise[num]
mask=mask[num]


In [ ]:
'''
ewsps
'''

%time p1 = ewfit(templates, galaxy, noise, velscale, start,Z=Z, moments=moments ,degree=-1,mdegree=-1,dust=None,linear_method='lsq_box',plot=False,clean=False, lam=lam, lam_temp=lam_temp, component=component,gas_component=gas_component, gas_names=gas_names, tie_balmer= tie_balmer, limit_doublets=limit_doublets,l=1e-8)

In [ ]:
dust=  [{"start": [0.5,1.3], "bounds": [[0, 2], [0.5,1.5]], "func":att_fun, "component": ~gas_component,"fixed":[False,False]}]
%time p2=pp(templates, galaxy, noise, velscale, start, moments=moments,  degree=-1, mdegree=-1, dust=dust, linear_method='lsq_box', plot=False, clean=False, lam=lam, lam_temp=lam_temp, component=component, gas_component=gas_component, gas_names=gas_names,quiet=False)

In [ ]:
p3=pp(templates, galaxy, noise, velscale, start, moments=moments, degree=-1 ,mdegree=10,dust=None,linear_method='lsq_box',plot=False,clean=False,lam=lam, lam_temp=lam_temp, component=component,gas_component=gas_component, gas_names=gas_names)

In [ ]:
sps_bc03.mean_age_metal(weights[:n_star].reshape(n_age,n_metal))
sps_bc03.mean_age_metal(p1.weights[:n_temps].reshape(n_age,n_metal))
print(mean_error(p1.tmp, p1.galaxy, p1.noise, p1.weights, template.age_grid), mean_error(p1.tmp, p1.galaxy, p1.noise, p1.weights, template.metal_grid))
sps_bc03.mean_age_metal(p2.weights[:n_temps].reshape(n_age,n_metal))
print(mean_error(p2.matrix.T, p2.galaxy, p2.noise, p2.weights, template.age_grid), mean_error(p2.matrix.T, p2.galaxy, p2.noise, p2.weights, template.metal_grid))
sps_bc03.mean_age_metal(p3.weights[:n_temps].reshape(n_age,n_metal))
print(mean_error(p3.matrix.T, p3.galaxy, p3.noise, p3.weights, template.age_grid), mean_error(p3.matrix.T, p3.galaxy, p3.noise, p3.weights, template.metal_grid))

In [ ]:
plt.figure(figsize=(15,10))

plt.axes([0.07,0.58,0.9,0.37])
plt.title('full spectrum')
plt.plot(p1.lam,p1.galaxy,'grey',lw=3, label = 'final mock spectrum')
plt.plot(lam_mock,galaxy0_mock,'k',lw=3, label = 'mock intrinsic spectrum')
plt.plot(p1.lam,p1.gal_temp@p1.weights,'dodgerblue',ls='--',lw=2, dashes=(10,5),label='SEW spectrum')
# plt.plot(p1.lam,p1t.gal_temp@p1t.weights,'grey',label='SEW')
plt.plot(p1.lam,p1.gal_temp[:,~gas_component]@p2.weights[~gas_component]/np.sum(p2.weights[~gas_component])+p1.gal_temp[:,gas_component]@p2.weights[gas_component],'r--',lw=3,label='pPXF+curve')
plt.plot(p1.lam,p1.gal_temp[:,~gas_component]@p3.weights[~gas_component]/np.sum(p3.weights[~gas_component])+p1.gal_temp[:,gas_component]@p3.weights[gas_component],'m--',lw=3, dashes=(3,5),label='pPXF+poly')

plt.legend(fontsize=15)


a_cut=3700
b_cut=4150
if resp_type == 'sti':
    a_cut=4800
    b_cut=5250
plt.plot([a_cut,3750],[2,2.2],'k')
plt.plot([b_cut,6500],[2,2.2],'k')
plt.plot([a_cut,b_cut,b_cut,a_cut,a_cut],[0.5,0.5,2,2,0.5],'k')
plt.xlim(3500,9000)
plt.ylim(0.3,4.78)
plt.xlabel(r'$\lambda(\AA)$')
plt.ylabel('normed flux')

ax=plt.axes([0.111,0.737,0.45,0.2])
ax.patch.set_alpha(0)
plt.plot(p1.lam,p1.galaxy,'grey',lw=5, label = 'final mock spectrum')
plt.plot(lam_mock,galaxy0_mock,'k',lw=5, label = 'mock intrinsic spectrum')
plt.plot(p1.lam,p1.gal_temp@p1.weights,'dodgerblue',ls='--',lw=3, dashes=(10,5),label='SEW spectrum')
plt.plot(p1.lam,p1.gal_temp[:,~gas_component]@p2.weights[~gas_component]/np.sum(p2.weights[~gas_component])+p1.gal_temp[:,gas_component]@p2.weights[gas_component],'r--',lw=3,label='pPXF+curve')
plt.plot(p1.lam,p1.gal_temp[:,~gas_component]@p3.weights[~gas_component]/np.sum(p3.weights[~gas_component])+p1.gal_temp[:,gas_component]@p3.weights[gas_component],'m--',lw=3, dashes=(3,5),label='pPXF+poly')



plt.xlim(a_cut,b_cut)
plt.ylim(0.5,2)
if resp_type == 'sti':
    plt.annotate('stitching point', xy=(5100, 1.1), xytext=(5100, 1.5), arrowprops=dict(facecolor='black', shrink=0.05))

plt.axes([0.07,0.08,0.9,0.37])
plt.title('attenuation curve')


ebv=0.3
ext = lambda x,delta:(x/5500)**-delta
att = lambda x, ebv, delta: ebv*(ext(x,delta)-1)/(((89/110)**-delta)-1)


plt.plot(p1.lam,att(p1.lam,ebv,1.3),'k--',lw=5,label='mock attenuaiton')
plt.plot(p1.lam,resp(p1.lam),'k:',lw=5,label='bias')
plt.plot(p1.lam,att(p1.lam,ebv,1.3)+resp(p1.lam),'k',lw=5,label='mock attenuaiton+bias')
plt.plot(p1.lam, -2.5*np.log10(p1.galaxy/(p1.gal_temp@p1.weights)),'b.',label='SEW',alpha=0.3,zorder=0)
plt.plot(p1.lam, 2.5*np.log10((p1.matrix_l@p1.weights)/p1.galaxy_l),'dodgerblue',ls='--',lw=5,label='smoothed SEW')
plt.plot(p2.lam,-2.5*np.log10(p2.dust[0]['func'](p2.lam,*p2.dust[0]['sol'])/p2.dust[0]['func'](5500,*p2.dust[0]['sol'])),'r--',lw=3,label='pPXF+curve')
plt.plot(p3.lam,-2.5*np.log10(p3.polyval(np.linspace(-1, 1, p3.npix), np.append(1.0, p3.mpolyweights)).clip(0.1)),'m--',lw=3,label='pPXF+poly')


plt.xlabel(r'$\lambda(\AA)$')
plt.ylabel(r'$A_\lambda-A_V$') 
plt.xlim(3550,9000)
plt.ylim(-0.7,1.)
plt.legend(ncol=2,fontsize=15)

plt.tight_layout()

mock stitching artifacts

In [ ]:
"""
initial a mock galaxy
"""
n_star=temp.shape[1] 

lam_range=np.array([3300,9000])

wave = np.array([3798.983, 3836.479, 3890.158, 3971.202, 4102.899, 4341.691, 4862.691, 6564.632])  # vacuum wavelengths
wave1=np.array([3726.032, 3728.815, 6716.44 , 6730.816, 3967.467, 3868.763,4685.704, 5875.614, 5006.843, 6300.298, 6583.452])
ratios = np.array([ 0.0530, 0.0731, 0.105, 0.159, 0.259, 0.468, 1, 2.86]) # [0.0530, 0.0731, 0.105, 0.159,]
ratios *= wave[-2]/wave  # Account for varying log-sampled pixel size in Angstrom
num0=np.where((wave>lam_range[0])&(wave<lam_range[1]))
num1=np.where((wave1>lam_range[0])&(wave1<lam_range[1]))

weights=np.zeros(n_star+num0[0].shape[0]+num1[0].shape[0])

weights[n_star:n_star+num0[0].shape[0]]=5*ratios[num0]
weights[n_star+num0[0].shape[0]:]=2*np.ones(num1[0].shape[0])#np.random.random(size=11)
# weights[120:]=0
num=np.arange(0,n_star)
np.random.seed(0)
np.random.shuffle(num)
np.random.seed()
weights[num[:10]]=1./10
comp = np.array([0]*n_star + [1]*8 + [2]*11)
gas = np.array(comp) >0
params =np.array([50, 80, -10, 100, 30, 50])
ebv=0.3
k=0.
ebv1=k*ebv
sn=30
delta=1.3
temp_file=os.path.join(os.getcwd(),'sps_models/BC03_SSP_tpl_chab.fits')
AGN=0.
resp_type='sti'
T=5100
del_A=0.05
if resp_type == 'sti':
    resp = lambda lam: -del_A*(np.sign(lam-T)+np.sign(T-5500))
elif resp_type == 'cal':
    resp = lambda lam: -del_A*np.sin(2*np.pi*(lam-5500)/T)
elif resp_type is None:
    resp = lambda lam: 0*lam

lam_range=np.array([3300,9000])
np.random.seed(0)
lam_mock,galaxy1_mock,galaxy0_mock,galaxy1_sn_mock, galaxy_emm_mock,lam_temp0,temp0,velscale,noise=mock_file(temp_file,lam_range,weights,comp,params, gas,ebv,ebv1,sn,v_delta=v_delta,delta=delta,savefits=True,AGN=AGN,resp=resp)
if AGN>0:
    weights=np.insert(weights,120,AGN)
    weights[:121]/=1+AGN

In [ ]:
from ppxf_sew import all_temp_make
name='mock.fits'

file=os.path.join(os.getcwd(),'test_data',name)

c = 299792.458

#data read
hdu=fits.open(file)
velscale=c*v_delta*np.log(10)
galaxy=hdu[0].data[0] #galaxy spectrum
galaxy0=hdu[0].data[1] #galaxy spectrum
noise=hdu[0].data[2] #galaxy noise
Z=hdu[0].header['Z'] # galaxy redshift
s=galaxy.shape[0]
mask=np.zeros(s) #galaxy mask
lam=(10**(hdu[0].header['COEFF0']+v_delta*np.arange(s)))/(1+Z) #galaxy lam

# temp read

from ppxf_sew import ew_util as lib


temp_file=os.path.join(os.getcwd(),'sps_models/BC03_SSP_tpl_chab.fits')
template=lib.temp(temp_file,velscale=velscale,FWHM_gal=2.76,FWHM_tem=2.5,normalize=True) #stellar template

temp=template.templates.reshape(template.templates.shape[0],-1)
lam_temp=np.exp(template.log_lam_temp)
lam_range_gal = np.array([np.min(lam), np.max(lam)])

AGN=0.

n_star=temp.shape[1] 


templates,start,moments,component,gas_component,gas_names,gas_wave=all_temp_make(temp,lam_temp,lam_range_gal)
if AGN>0:
    templates=np.insert(templates,120,power(lam_temp),axis=1)
vlim = np.array([-2900, 2900]) 
lam_range = lam_temp[[0, -1]]/np.exp(vlim/c)
num=(lam > lam_range[0]) & (lam < lam_range[1])
lam=lam[num]
galaxy=galaxy[num]

galaxy1=galaxy1_mock[num]
galaxy0=galaxy0_mock[num]
noise=noise[num]
mask=mask[num]


In [ ]:
%time p1 = ewfit(templates, galaxy, noise, velscale, start,Z=Z, moments=moments ,degree=-1,mdegree=-1,dust=None,linear_method='lsq_box',plot=False,clean=False, lam=lam, lam_temp=lam_temp, component=component,gas_component=gas_component, gas_names=gas_names, tie_balmer= tie_balmer, limit_doublets=limit_doublets,l=1e-8)

In [ ]:
dust=  [{"start": [0.5,1.3], "bounds": [[0, 2], [0.5,1.5]], "func":att_fun, "component": ~gas_component,"fixed":[False,False]}]
%time p2=pp(templates, galaxy, noise, velscale, start, moments=moments,  degree=-1, mdegree=-1, dust=dust, linear_method='lsq_box', plot=False, clean=False, lam=lam, lam_temp=lam_temp, component=component, gas_component=gas_component, gas_names=gas_names,quiet=False)

In [ ]:
p3=pp(templates, galaxy, noise, velscale, start, moments=moments, degree=-1 ,mdegree=10,dust=None,linear=False, linear_method='lsq_box',plot=False,clean=False,lam=lam, lam_temp=lam_temp, component=component,gas_component=gas_component, gas_names=gas_names)

In [ ]:
sps_bc03.mean_age_metal(weights[:n_star].reshape(n_age,n_metal))
sps_bc03.mean_age_metal(p1.weights[:n_temps].reshape(n_age,n_metal))
print(mean_error(p1.tmp, p1.galaxy, p1.noise, p1.weights, template.age_grid), mean_error(p1.tmp, p1.galaxy, p1.noise, p1.weights, template.metal_grid))
sps_bc03.mean_age_metal(p2.weights[:n_temps].reshape(n_age,n_metal))
print(mean_error(p2.matrix.T, p2.galaxy, p2.noise, p2.weights, template.age_grid), mean_error(p2.matrix.T, p2.galaxy, p2.noise, p2.weights, template.metal_grid))
sps_bc03.mean_age_metal(p3.weights[:n_temps].reshape(n_age,n_metal))
print(mean_error(p3.matrix.T, p3.galaxy, p3.noise, p3.weights, template.age_grid), mean_error(p3.matrix.T, p3.galaxy, p3.noise, p3.weights, template.metal_grid))

In [ ]:
plt.figure(figsize=(15,10))

plt.axes([0.07,0.58,0.9,0.37])
plt.title('full spectrum')
plt.plot(p1.lam,p1.galaxy,'grey',lw=3, label = 'final mock spectrum')
plt.plot(lam_mock,galaxy0_mock,'k',lw=3, label = 'mock intrinsic spectrum')
plt.plot(p1.lam,p1.gal_temp@p1.weights,'dodgerblue',ls='--',lw=2, dashes=(10,5),label='SEW spectrum')
# plt.plot(p1.lam,p1t.gal_temp@p1t.weights,'grey',label='SEW')
plt.plot(p1.lam,p1.gal_temp[:,~gas_component]@p2.weights[~gas_component]/np.sum(p2.weights[~gas_component])+p1.gal_temp[:,gas_component]@p2.weights[gas_component],'r--',lw=3, dashes=(3,5),label='pPXF+curve')
plt.plot(p1.lam,p1.gal_temp[:,~gas_component]@p3.weights[~gas_component]/np.sum(p3.weights[~gas_component])+p1.gal_temp[:,gas_component]@p3.weights[gas_component],'m--',lw=3,label='pPXF+poly')

# plt.plot(p1.lam,p1.bestfit,'dodgerblue',ls='--',lw=3, dashes=(10,5),label='SEW spectrum')
# plt.plot(p1.lam,p2.bestfit,'r--',lw=3,label='pPXF+curve')
# plt.plot(p1.lam,p3.bestfit,'m--',lw=3, dashes=(3,5),label='pPXF+poly')
plt.legend(fontsize=15)


a_cut=3700
b_cut=4150
if resp_type == 'sti':
    a_cut=T-200
    b_cut=T+200
plt.plot([a_cut,3750],[2,2.2],'k')
plt.plot([b_cut,6500],[2,2.2],'k')
plt.plot([a_cut,b_cut,b_cut,a_cut,a_cut],[0.5,0.5,2,2,0.5],'k')
plt.xlim(3500,9000)
plt.ylim(0.3,4.78)
plt.xlabel(r'$\lambda(\AA)$')
plt.ylabel('normed flux')

ax=plt.axes([0.111,0.737,0.45,0.2])
ax.patch.set_alpha(0)
plt.plot(p1.lam,p1.galaxy,'grey',lw=5, label = 'final mock spectrum')
plt.plot(lam_mock,galaxy0_mock,'k',lw=5, label = 'mock intrinsic spectrum')
plt.plot(p1.lam,p1.gal_temp@p1.weights,'dodgerblue',ls='--',lw=3, dashes=(10,5),label='SEW spectrum')
plt.plot(p1.lam,p1.gal_temp[:,~gas_component]@p2.weights[~gas_component]/np.sum(p2.weights[~gas_component])+p1.gal_temp[:,gas_component]@p2.weights[gas_component],'r--',lw=3, dashes=(3,5),label='pPXF+curve')
plt.plot(p1.lam,p1.gal_temp[:,~gas_component]@p3.weights[~gas_component]/np.sum(p3.weights[~gas_component])+p1.gal_temp[:,gas_component]@p3.weights[gas_component],'m--',lw=3,label='pPXF+poly')



# plt.plot(p1.lam,p1.bestfit,'dodgerblue',ls='--',lw=3, dashes=(10,5),label='SEW spectrum')
# plt.plot(p1.lam,p2.bestfit,'r--',lw=3,label='pPXF+curve')
# plt.plot(p1.lam,p3.bestfit,'m--',lw=3, dashes=(3,5),label='pPXF+poly')



plt.xlim(a_cut,b_cut)
plt.ylim(0.5,2)
if resp_type == 'sti':
    plt.annotate('stitching point', xy=(T, 1.1), xytext=(T, 1.5), arrowprops=dict(facecolor='black', shrink=0.05))

plt.axes([0.07,0.08,0.9,0.37])
plt.title('attenuation curve')


ebv=0.3
ext = lambda x,delta:(x/5500)**-delta
att = lambda x, ebv, delta: ebv*(ext(x,delta)-1)/(((89/110)**-delta)-1)




plt.plot(p1.lam,att(p1.lam,ebv,1.3),'k--',lw=5,label='mock attenuaiton')
plt.plot(p1.lam,resp(p1.lam),'k:',lw=5,label='bias')
plt.plot(p1.lam,att(p1.lam,ebv,1.3)+resp(p1.lam),'k',lw=5,label='mock attenuaiton+bias')
plt.plot(p1.lam, -2.5*np.log10(p1.galaxy/(p1.gal_temp@p1.weights)),'b.',label='SEW',alpha=0.3,zorder=0)
plt.plot(p1.lam, 2.5*np.log10((p1.matrix_l@p1.weights)/p1.galaxy_l),'dodgerblue',ls='--',lw=5,label='smoothed SEW')
plt.plot(p2.lam,-2.5*np.log10(p2.dust[0]['func'](p2.lam,*p2.dust[0]['sol'])/p2.dust[0]['func'](5500,*p2.dust[0]['sol'])),'r--',lw=3,label='pPXF+curve')
plt.plot(p3.lam,-2.5*np.log10(p3.polyval(np.linspace(-1, 1, p3.npix), np.append(1.0, p3.mpolyweights)).clip(0.1)),'m--',lw=3,label='pPXF+poly')


plt.xlabel(r'$\lambda(\AA)$')
plt.ylabel(r'$A_\lambda-A_V$')
plt.xlim(3550,9000)
plt.ylim(-0.7,1.)
plt.legend(ncol=2,fontsize=15)

plt.tight_layout()

statistical tests

Warning: This test will take about 30-40 CPU hours, we recommend doing this test on a server!
Note: Due to random number effects, the test results here may be little different from those in the paper.

In [ ]:
"""
initial test environment
"""
from ppxf_sew import all_temp_make
from scipy.optimize import curve_fit,leastsq
from tqdm import trange
from ppxf.ppxf import ppxf as pp


def test_ewfit(j,ebv,delta,sn,v_delta=1e-4,l=1e-8,p=0.5,AGN=0,resp_type='sti', del_A = 0.02, T = 1500):
    rat=[0,1,10,100]
    n_star=120
    lam_range=np.array([3300,9000])
    
    wave = np.array([3798.983, 3836.479, 3890.158, 3971.202, 4102.899, 4341.691, 4862.691, 6564.632])  # vacuum wavelengths
    wave1=np.array([3726.032, 3728.815, 6716.44 , 6730.816, 3967.467, 3868.763,4685.704, 5875.614, 5006.843, 6300.298, 6583.452])
    ratios = np.array([ 0.0530, 0.0731, 0.105, 0.159, 0.259, 0.468, 1, 2.86]) # [0.0530, 0.0731, 0.105, 0.159,]
    ratios *= wave[-2]/wave  # Account for varying log-sampled pixel size in Angstrom
    num0=np.where((wave>lam_range[0])&(wave<lam_range[1]))
    num1=np.where((wave1>lam_range[0])&(wave1<lam_range[1]))
    q=1
    weights=np.zeros(n_star+num0[0].shape[0]+num1[0].shape[0])

    num=np.arange(0,n_star)
    np.random.seed(j)
    np.random.shuffle(num)
    np.random.seed()
    weights[num[:10]]=1./10
    comp = np.array([0]*n_star + [1]*8 + [2]*11)
    gas = np.array(comp) >0
    params =np.array([50, 80, -10, 100, 30, 50])
    k=0
  

    temp_file=os.path.join(os.getcwd(),'sps_models/BC03_SSP_tpl_chab.fits')

    if resp_type == 'sti':
        resp = lambda lam: -0.05*(np.sign(lam-5100)-1)
    elif resp_type == 'cal':
        resp = lambda lam: -del_A*np.sin(2*np.pi*(lam-5500)/T)
    elif resp_type is None:
        resp = lambda lam: 0*lam

    lam,galaxy1,galaxy0,galaxy1_sn, galaxy_emm,lam_temp0,temp0,velscale,noise=mock_file(temp_file,lam_range,weights,comp,params,gas,ebv,k,sn,v_delta=v_delta,delta=delta,AGN=AGN,resp=resp)
    if AGN>0:
        weights=np.insert(weights,120,AGN)
        weights[:121]/=1+AGN


    c = 299792.458


    galaxy=galaxy1_sn

    noise=noise
    Z=0

    s=galaxy.shape[0]
    mask=np.zeros(s) #galaxy mask



    template=lib.temp(temp_file,velscale=velscale,FWHM_gal=2.76,FWHM_tem=2.5,normalize=True) #stellar template

    temp=template.templates.reshape(template.templates.shape[0],-1)
    lam_temp=np.exp(template.log_lam_temp)
    lam_range_gal = np.array([np.min(lam), np.max(lam)])

    templates,start,moments,component,gas_component,gas_names=all_temp_make(temp,lam_temp,lam_range_gal, tie_balmer=False, limit_doublets=False)
    # print(start)
    start = np.array([[50, 80], [-10, 100], [30, 50]])
    vlim = np.array([-2900, 2900])  # As 2e3 nonlinear_fit() +900 for 3sigma
    lam_range = lam_temp[[0, -1]]/np.exp(vlim/c)
    num=(lam > lam_range[0]) & (lam < lam_range[1])
    lam=lam[num]
    galaxy=galaxy[num]
    galaxy0=galaxy0[num]
    galaxy1=galaxy1[num]
    noise=noise[num]
    mask=mask[num]



    att_fun0=partial(att_fun,resp=resp)
    att_fun1 = partial(att_fun,resp=None)
    
    ext = lambda x,delta:(x/5500)**-delta
    att = lambda x, ebv, delta : 2.5*np.log10(att_fun0(np.array([5500]),ebv,delta))-2.5*np.log10(att_fun0(x,ebv,delta))

    if AGN>0:
        templates=np.insert(templates, 120, power(lam_temp),  axis=1)
        component=np.insert(component,120,0)
        gas_component=np.insert(gas_component,120,0)
    lin_method='cvxopt'#'lsq_box' #
    try:
        p1 = ewfit(templates, galaxy, noise, velscale, start,Z=Z, moments=moments ,dust=None,linear=True, linear_method=lin_method,plot=False,clean=False, degree=-1,lam=lam, lam_temp=lam_temp, component=component,gas_component=gas_component, gas_names=gas_names, tie_balmer= tie_balmer, limit_doublets=limit_doublets,l=l,p=p,quiet=True)
    except:
        p1 = ewfit(templates, galaxy, noise, velscale, start,Z=Z, moments=moments ,dust=None,linear=True,linear_method='lsq_box',plot=False,clean=False, degree=-1,lam=lam, lam_temp=lam_temp, component=component,gas_component=gas_component, gas_names=gas_names, tie_balmer= tie_balmer, limit_doublets=limit_doublets,l=l,p=p,quiet=True)

    young_comp=np.array([1]*30+[0]*(139-30),dtype=bool)
    # if AGN>0:
    #     gas_component[120]=1
    #     gas_names=np.insert(gas_names,0,'AGN')

    

    dust0= [{"start": [0.3,1.3], "bounds": [[0, 2], [0.5,1.5]], "func":att_fun1, "component": ~gas_component,"fixed":[False,False]}]
    dust1= [{"start": [0.3], "bounds": [[0, 2]], "component": ~gas_component,"fixed":[False]}]
    dust2= [{"start": [0.3,1.3], "bounds": [[0, 2], [0.5,1.5]], "func":att_fun0, "component": ~gas_component,"fixed":[False,False]}]
    try: #ppxf+power
        p2=pp(templates, galaxy, noise, velscale, start, moments=moments, dust=dust0, linear=True,linear_method=lin_method, plot=False, clean=False, degree=-1,lam=lam, lam_temp=lam_temp, component=component, gas_component=gas_component, gas_names=gas_names,quiet=True)
    except:
        p2=pp(templates, galaxy, noise, velscale, start, moments=moments, dust=dust0, linear=True,linear_method='lsq_box', plot=False, clean=False, degree=-1,lam=lam, lam_temp=lam_temp, component=component, gas_component=gas_component, gas_names=gas_names,quiet=True)
    try: #ppxf+calzetti
        p3=pp(templates, galaxy, noise, velscale, start, moments=moments, dust=dust1, linear=True,linear_method=lin_method, plot=False, clean=False, degree=-1,lam=lam, lam_temp=lam_temp, component=component, gas_component=gas_component, gas_names=gas_names,quiet=True)
    except:
        p3=pp(templates, galaxy, noise, velscale, start, moments=moments, dust=dust1, linear=True,linear_method='lsq_box', plot=False, clean=False, degree=-1,lam=lam, lam_temp=lam_temp, component=component, gas_component=gas_component, gas_names=gas_names,quiet=True)
    try: #ppxf+true
        p4=pp(templates, galaxy, noise, velscale, start, moments=moments, dust=dust2, linear=True,linear_method=lin_method, plot=False, clean=False, degree=-1,lam=lam, lam_temp=lam_temp, component=component, gas_component=gas_component, gas_names=gas_names,quiet=True)
    except:
        p4=pp(templates, galaxy, noise, velscale, start, moments=moments, dust=dust2, linear=True,linear_method='lsq_box', plot=False, clean=False, degree=-1,lam=lam, lam_temp=lam_temp, component=component, gas_component=gas_component, gas_names=gas_names,quiet=True)

    try: #ppxf+poly
        p5=pp(templates, galaxy, noise, velscale, start, moments=moments, dust=None, linear=False,linear_method=lin_method, plot=False, clean=False, degree=-1, mdegree=10,lam=lam, lam_temp=lam_temp, component=component, gas_component=gas_component, gas_names=gas_names,quiet=True)
    except:
        p5=pp(templates, galaxy, noise, velscale, start, moments=moments, dust=None, linear=False,linear_method='lsq_box', plot=False, clean=False, degree=-1, mdegree=10,lam=lam, lam_temp=lam_temp, component=component, gas_component=gas_component, gas_names=gas_names,quiet=True)
    # plt.plot(p1.lam,p1.att_curve,'.')
    # plt.plot(p1.lam,att(p1.lam,ebv,delta),'--')
    # err= lambda p,x,y: att(x,*p)-y
    num=np.where(p1.att_curve==p1.att_curve)
    p1att=curve_fit(att,p1.lam[num], p1.att_curve[num],p0=[ebv,delta],bounds=((-0.1,0.5),(1,2)))[0]
    # curve=p1.att_curve_smooth
    # flux=p1.weights[120:]
    # except:
    #     data=np.array([0,0,j])
    #     curve=lam
    age1,metal1=template.mean_age_metal(p1.weights[:120].reshape(20,6),quiet=True)
    age2,metal2=template.mean_age_metal(p2.weights[:120].reshape(20,6),quiet=True)
    age3,metal3=template.mean_age_metal(p3.weights[:120].reshape(20,6),quiet=True)
    age4,metal4=template.mean_age_metal(p4.weights[:120].reshape(20,6),quiet=True)
    age5,metal5=template.mean_age_metal(p5.weights[:120].reshape(20,6),quiet=True)
    age0,metal0=template.mean_age_metal(weights[:120].reshape(20,6),quiet=True) 
    return np.array([[age0,metal0],[age1,metal1],[age2,metal2],[age3,metal3],[age4,metal4],[age5,metal5]]),np.array([p1att, p2.dust[0]['sol'],np.array([p3.dust[0]['sol'][0]/4.1,1.3]),p4.dust[0]['sol']]) ,p1,lam,galaxy0,galaxy1,weights,np.array([age0,metal0])
    # return np.array([age,metal]),p1,weights

In [ ]:
from tqdm import trange
from multiprocessing import cpu_count
np.random.seed(0)
size=1000
ls=np.random.randint(1000000,size=size)


l_list=np.array([1e5,1e6,1e7,1e8])
ebv=0.3
delta=1.3
sn_list=[5,15,30]
resp_list=[None,'cal','sti']
data=np.zeros((len(resp_list),len(sn_list),size,6,2))
data0=np.zeros((len(resp_list),len(sn_list),size,2))
dust=np.zeros((len(resp_list),len(sn_list),size,4,2))


In [ ]:

pool=Pool(processes=min(cpu_count(),100))

T_list=[1500,1000,5100]
for r in range(3):

    for l in range(3):
        fun0=partial(test_ewfit,ebv=ebv,v_delta=1e-4,delta=delta,sn=sn_list[l],l=1e-8,p=0.5,AGN=0.,resp_type=resp_list[r],del_A=0.05, T=T_list[r])
        res0= [pool.apply_async(fun0,args=(ls[k],)) for k in range(size)]
        for k in trange(size): 
            data[r,l,k],dust[r,l,k],p1,lam,galaxy0,galaxy1,weights,data0[r,l,k] = res0[k].get() 

pool.close()
pool.join()

In [ ]:
color=['dodgerblue','r','m','grey','orange','c','y','k','orange','purple','brown']
marker=['o','s','^','D','v']
p=0
bias_list=['no bias','calibration bias','stitching bias']
labels=['SEW','pPXF+curve','pPXF+misfit','pPXF+accurate','pPXF+poly']
fig, axes = plt.subplots(3,3,figsize=(15,12), sharex=True, sharey=True)


for r in range(3):
    
    for l in range(3):
        for p in range(5):
            if r==0 and p in [1,4]:
                continue
            elif r>0 and p== 2:
                continue
            else:
                axes[r,l].scatter(np.mean(data[r,l,:,p+1,0]-data[r,l,:,0,0]),np.mean(data[r,l,:,p+1,1]-data[r,l,:,0,1]),c=color[p],marker = marker[p],label=labels[p],zorder=10)
                print(f'S/N={sn_list[l]},'+bias_list[r]+','+labels[p]+':(%s,%s),(%s,%s)'%(round(np.mean(data[r,l,:,p+1,0]-data[r,l,:,0,0]),2),round(np.std(data[r,l,:,p+1,0]-data[r,l,:,0,0]),2),round(np.mean(data[r,l,:,p+1,1]-data[r,l,:,0,1]),2),round(np.std(data[r,l,:,p+1,1]-data[r,l,:,0,1]),2)))
                kdeplot(x=data[r,l,:,p+1,0]-data[r,l,:,0,0],y=data[r,l,:,p+1,1]-data[r,l,:,0,1],shade=False,color=color[p],levels=[0.25,0.5,0.75],ax=axes[r,l])
        axes[r,l].set_xlim(-1.,1.)
        axes[r,l].set_ylim(-1.,1.)
        axes[r,l].plot(0,0,'kx',ms = 15,zorder=5)
        axes[r,l].text(-0.9,0.8,f'S/N={sn_list[l]},'+bias_list[r], fontsize=20)

fig.text(0,0.5,r'$\Delta [Z/Z_\odot]$',va='center',rotation='vertical')
fig.text(0.5,0,r'$\Delta \log age_L$',ha='center')
lines1,labels1=fig.axes[0].get_legend_handles_labels()
lines2,labels2=fig.axes[-1].get_legend_handles_labels()
lines=[lines1[0],lines1[2],lines1[1],lines2[1],lines2[3]]
labels=[labels1[0],labels1[2],labels1[1],labels2[1],labels2[3]]
fig.legend(lines,labels,ncol=5,loc='lower center',frameon=True, bbox_to_anchor=(0.5, 0.98))
plt.tight_layout()



Appendix B1

In [ ]:
color=['dodgerblue','r','m','grey','orange','c','y','k','orange','purple','brown']
marker=['o','s','^','D','v']


from matplotlib.collections import PathCollection
from matplotlib.legend_handler import HandlerLine2D
def updateline(handle, orig):
    handle.update_from(orig)
    handle.set_markersize(20)
    handle.set_alpha(1)
# for p in range(4):
p=0
bias_list=['no bias','calibration bias','stitching bias']
labels=['SEW','pPXF+curve','pPXF+misfit','pPXF+accurate','pPXF+poly']
fig, axes = plt.subplots(3,3,figsize=(15,10), sharex=True, sharey=True)
order_list=[4,0,1,3,2]
for r in range(3): 
    for l in range(3):
        for p in range(5):
            if r==0 and p in [1,4]:
                continue
            elif r>0 and p== 2:
                continue
            else:
                axes[r,l].plot(data[r,l,:,0,0],data[r,l,:,p+1,0],'.',c=color[p],label=labels[p],ms=3,zorder=order_list[p],alpha=0.3)
        # axes[r,l].plot(np.linspace(7,10),np.linspace(7,10),'k--')
        axes[r,l].plot(np.linspace(7,10),np.linspace(7,10),'k--')
        axes[r,l].set_xlim(6.8,10.2)
        axes[r,l].set_ylim(6.8,10.2)
        axes[r,l].text(7,9.8,f'S/N={sn_list[l]},'+bias_list[r], fontsize=20)

fig.text(0,0.5,r'$\log age_L$ (Fitted)',va='center',rotation='vertical')
fig.text(0.5,0,r'$\log age_L$ (True)',ha='center')
lines1,labels1=fig.axes[0].get_legend_handles_labels()
lines2,labels2=fig.axes[-1].get_legend_handles_labels()
lines=[lines1[0],lines1[2],lines1[1],lines2[1],lines2[3]]
labels=[labels1[0],labels1[2],labels1[1],labels2[1],labels2[3]]
fig.legend(lines,labels,ncol=5,loc='lower center',frameon=True, bbox_to_anchor=(0.5, 0.98),handler_map={plt.Line2D: HandlerLine2D(update_func=updateline)},)
plt.tight_layout()

Appendix B2

In [ ]:

color=['dodgerblue','r','m','grey','orange','c','y','k','orange','purple','brown']
marker=['o','s','^','D','v']
# for p in range(4):
p=0
bias_list=['no bias','calibration bias','stitching bias']
labels=['SEW','pPXF+curve','pPXF+misfit','pPXF+accurate','pPXF+poly']
fig, axes = plt.subplots(3,3,figsize=(15,10), sharex=True, sharey=True)
order_list=[4,0,1,3,2]
for r in range(3): 
    for l in range(3):
        for p in range(5):
            if r==0 and p in [1,4]:
                continue
            elif r>0 and p== 2:
                continue
            else:
                axes[r,l].plot(data[r,l,:,0,1],data[r,l,:,p+1,1],'.',c=color[p],label=labels[p],ms=3,zorder=order_list[p],alpha=0.3)
        # axes[r,l].plot(np.linspace(7,10),np.linspace(7,10),'k--')
        axes[r,l].plot(np.linspace(-1.9,0.1),np.linspace(-1.9,0.1),'k--')
        axes[r,l].set_xlim(-2,0.2)
        axes[r,l].set_ylim(-2,0.2)
        axes[r,l].text(-1.8,0,f'S/N={sn_list[l]},'+bias_list[r], fontsize=20)

fig.text(0,0.5,r'$[Z/Z_\odot]$ (Fitted)',va='center',rotation='vertical')
fig.text(0.5,0,r'$[Z/Z_\odot]$ (True)',ha='center')
lines1,labels1=fig.axes[0].get_legend_handles_labels()
lines2,labels2=fig.axes[-1].get_legend_handles_labels()
lines=[lines1[0],lines1[2],lines1[1],lines2[1],lines2[3]]
labels=[labels1[0],labels1[2],labels1[1],labels2[1],labels2[3]]
fig.legend(lines,labels,ncol=5,loc='lower center',frameon=True, bbox_to_anchor=(0.5, 0.98),handler_map={plt.Line2D: HandlerLine2D(update_func=updateline)},)
plt.tight_layout()

Figure 5
Warning: This test will also take about 30-40 CPU hours, we recommend doing this test on a server!
Note: Due to random number effects, the test results here may be little different from those in the paper.

In [ ]:
from tqdm import trange
np.random.seed(11)
size=1000
ls=np.random.randint(1000000,size=size)


l_list=np.array([1e5,1e6,1e7,1e8])
# p_list=[0.1,0.3,0.5,0.7,0.9]
ebv=0.3
delta=1.3
sn_list=[5,15,30]
resp_list=['sti','cal']
data=np.zeros((3,5,size,6,2))
data0=np.zeros((3,5,size,2))
dust=np.zeros((3,5,size,4,2))

In [ ]:
pool=Pool(processes=min(cpu_count(),64))
R=np.array([100,500,1500,4500])
v_delta_list=1./R/np.log(10)
l_list=np.array([1e-4,1e-6,1e-8,1e-10,1e-12])
for r in range(3):
    for l in range(5):
        fun0=partial(test_ewfit,ebv=ebv,v_delta=1e-4,delta=delta,sn=sn_list[r],l=l_list[l],p=0.5,AGN=0., resp=None)
        res0= [pool.apply_async(fun0,args=(ls[k],)) for k in range(size)]
        for k in trange(size): 
            data[r,l,k],dust[r,l,k],p1,lam,galaxy0,galaxy1,weights,data0[r,l,k] = res0[k].get() 

In [ ]:
color=['dodgerblue','r','g','grey','c','m','y','k','orange','purple','brown']
marker=['o','s','^','D','v']
# for p in range(4):
p=0
bias_list=['no bias','calibration bias','stitching bias']
labels=['SEW','pPXF+power-law','pPXF+calzetti','pPXF+accurate']
# fig=plt.figure(figsize=(20,15))
fig, axes = plt.subplots(3,5,figsize=(20,10), sharex=True, sharey=True)
for r in range(3):
    for l in range(5):
        for p in range(4):
            if p in [1,2]:
                continue
            else:
                axes[r,l].scatter(np.mean(data[r,l,:,p+1,0]-data[r,l,:,0,0]),np.mean(data[r,l,:,p+1,1]-data[r,l,:,0,1]),c=color[p],marker = marker[p],label=labels[p],zorder=10)
                axes[r,l].errorbar(np.mean(data[r,l,:,p+1,0]-data[r,l,:,0,0]),np.mean(data[r,l,:,p+1,1]-data[r,l,:,0,1]),xerr=np.std(data[r,l,:,p+1,0]-data[r,l,:,0,0])/np.sqrt(size),yerr=np.std(data[r,l,:,p+1,1]-data[r,l,:,0,1])/np.sqrt(size),c=color[p],marker=marker[p],zorder=0)
                kdeplot(x=data[r,l,:,p+1,0]-data[r,l,:,0,0],y=data[r,l,:,p+1,1]-data[r,l,:,0,1],shade=False,color=color[p],levels=[0.25,0.5,0.75],ax=axes[r,l])

        axes[r,l].plot(0,0,'kx',ms = 15,zorder=5)
        
        axes[r,l].set_xlim(-0.8,0.8)
        axes[r,l].set_ylim(-0.8,0.8)
        lam_dpls = f'{l_list[l]:.0e}'
        resolution=f'{1./(v_delta_list[r]*np.log(10)):.1f}'
        axes[r,l].text(-0.7,0.6,fr'$\lambda = {lam_dpls}, S/N = {sn_list[r]}$', fontsize=18)

fig.text(0,0.5,r'$\Delta [Z/Z_\odot]$',va='center',rotation='vertical')
fig.text(0.5,0,r'$\Delta \log age_L$',ha='center')
# plt.xlabel(r'$\Delta \log age_L$')
# plt.ylabel(r'$\Delta [Z/Z_\odot]$')
lines,labels=fig.axes[-1].get_legend_handles_labels()

fig.legend(lines,labels,ncol=4,loc='lower center',frameon=True, bbox_to_anchor=(0.5, 0.98))
plt.tight_layout()

CASE III

In [ ]:
"""
intrinsic a mock galaxy
"""
n_star=temp.shape[1] 

lam_range=np.array([3300,9000])

wave = np.array([3798.983, 3836.479, 3890.158, 3971.202, 4102.899, 4341.691, 4862.691, 6564.632])  # vacuum wavelengths
wave1=np.array([3726.032, 3728.815, 6716.44 , 6730.816, 3967.467, 3868.763,4685.704, 5875.614, 5006.843, 6300.298, 6583.452])
ratios = np.array([ 0.0530, 0.0731, 0.105, 0.159, 0.259, 0.468, 1, 2.86]) # [0.0530, 0.0731, 0.105, 0.159,]
ratios *= wave[-2]/wave  # Account for varying log-sampled pixel size in Angstrom
num0=np.where((wave>lam_range[0])&(wave<lam_range[1]))
num1=np.where((wave1>lam_range[0])&(wave1<lam_range[1]))

weights=np.zeros(n_star+num0[0].shape[0]+num1[0].shape[0])

weights[n_star:n_star+num0[0].shape[0]]=5*ratios[num0]
weights[n_star+num0[0].shape[0]:]=2*np.ones(num1[0].shape[0])#np.random.random(size=11)
# weights[120:]=0
num=np.arange(0,n_star)
np.random.seed(0)
np.random.shuffle(num)
# np.random.seed()
weights[num[:10]]=1./10
comp = np.array([0]*n_star + [1]*8 + [2]*11)
gas = np.array(comp) >0
params =np.array([50, 80, -10, 100, 30, 50])
ebv=0.3
k=0.5
ebv1=k*ebv
sn=30
delta=1.3
temp_file=os.path.join(os.getcwd(),'sps_models/BC03_SSP_tpl_chab.fits')
# temp_file=None
AGN=0.
resp=None
lam_range=np.array([3300,9000])
np.random.seed(10)
lam_mock,galaxy1_mock,galaxy0_mock,galaxy1_sn_mock, galaxy_emm_mock,lam_temp0,temp0,velscale,noise=mock_file(temp_file,lam_range,weights,comp,params, gas,ebv,ebv1,sn,v_delta=v_delta,delta=delta,savefits=True,AGN=AGN,resp=resp)
if AGN>0:
    weights=np.insert(weights,120,AGN)
    weights[:121]/=1+AGN
    



In [ ]:
from ppxf_sew import all_temp_make
name='mock.fits'

file=os.path.join(os.getcwd(),'test_data',name)

c = 299792.458

#data read
hdu=fits.open(file)
velscale=c*v_delta*np.log(10)
galaxy=hdu[0].data[0] #galaxy spectrum
galaxy0=hdu[0].data[1] #galaxy spectrum
noise=hdu[0].data[2] #galaxy noise
Z=hdu[0].header['Z'] # galaxy redshift
s=galaxy.shape[0]
mask=np.zeros(s) #galaxy mask
lam=(10**(hdu[0].header['COEFF0']+v_delta*np.arange(s)))/(1+Z) #galaxy lam

# temp read

from ppxf_sew import ew_util as lib


temp_file=os.path.join(os.getcwd(),'sps_models/BC03_SSP_tpl_chab.fits')
template=lib.temp(temp_file,velscale=velscale,FWHM_gal=2.76,FWHM_tem=2.5,normalize=True) #stellar template

temp=template.templates.reshape(template.templates.shape[0],-1)
lam_temp=np.exp(template.log_lam_temp)
lam_range_gal = np.array([np.min(lam), np.max(lam)])

AGN=0.

n_star=temp.shape[1] 


templates,start,moments,component,gas_component,gas_names,gas_wave=all_temp_make(temp,lam_temp,lam_range_gal)
if AGN>0:
    templates=np.insert(templates,120,power(lam_temp),axis=1)
# np.hstack([templates,np.array([power(lam_temp)]).T])
vlim = np.array([-2900, 2900])  # As 2e3 nonlinear_fit() +900 for 3sigma
lam_range = lam_temp[[0, -1]]/np.exp(vlim/c)
num=(lam > lam_range[0]) & (lam < lam_range[1])
lam=lam[num]
galaxy=galaxy[num]
# galaxy1=galaxy1[num]
# galaxy_emm=galaxy_emm[num]
galaxy1=galaxy1_mock[num]
galaxy0=galaxy0_mock[num]
noise=noise[num]
mask=mask[num]


In [ ]:
from ppxf.ppxf import ppxf as pp
ext = lambda x,delta:(x/5500)**-delta


def att_fun1(lam,ebv,delta,f_nodust=0):
    # delta=-np.log(1+1/Rv)/np.log(89/110)
    
    Rv=1/(((89/110)**-delta)-1)
    # k=(110.-89.**2/110./Rv)/21.
    # ext = lambda x:k*(x/5500)**-1+(1-k)*(x/5500)**-2
    frac = 10**(-0.4*ebv*Rv*ext(lam,delta).clip(0))
    frac = f_nodust + (1-f_nodust)*frac
    return frac

start = [0, 180.]
# params =np.array([50, 80, -10, 100, 30, 50])
n_temps = temp.shape[1] 
n_forbidden = np.sum(["[" in a or "HeI" in a for a in gas_names])
n_balmer = len(gas_names) - n_forbidden
component = np.array([0]*n_temps + [1]*n_balmer + [2]*n_forbidden)
if AGN>0:
    component = np.insert(component,120,0)
gas_component = np.array(component) >0
moments = [2,2,2]
start = [start, start[:2], start[:2]]

young_comp=np.array([1]*30+[0]*(139-30),dtype=bool)
old_comp=np.array([0]*30+[1]*90+[0]*(139-120),dtype=bool)




In [ ]:
'''
ewsps
'''

%time p1 = ewfit(templates, galaxy, noise, velscale, start,Z=Z, moments=moments ,degree=-1,mdegree=-1,dust=None,linear_method='lsq_box',plot=False,clean=False, lam=lam, lam_temp=lam_temp, component=component,gas_component=gas_component, gas_names=gas_names, tie_balmer= tie_balmer, limit_doublets=limit_doublets,l=1e-8)

In [ ]:
dustt= [{"start": [0.1,1.3], "bounds": [[-0.2,0.2], [0.5,1.5]], "func":att_fun1, "component": young_comp,"fixed":[False,False]}]
%time p1t = ewfit(templates, galaxy, noise, velscale, start,Z=Z, moments=moments ,degree=-1,dust=dustt,linear_method='lsq_box',plot=False,clean=False, lam=lam, lam_temp=lam_temp, component=component,gas_component=gas_component, gas_names=gas_names, tie_balmer= tie_balmer, limit_doublets=limit_doublets,l=1e-8)

In [ ]:
plt.figure(figsize=(15,10))

plt.axes([0.07,0.58,0.9,0.37])
plt.title('full spectrum')
plt.plot(p1.lam,p1.galaxy,'grey',lw=3, label = 'input spectrum')
plt.plot(lam_mock,galaxy0_mock,'k',lw=3, label = 'mock intrinsic spectrum')
plt.plot(p1.lam,p1.gal_temp@p1.weights,'dodgerblue',ls='--',lw=2, dashes=(10,5),label='SEW spectrum')
plt.plot(p1.lam,p1.gal_temp@p1t.weights,'r',ls='-.',lw=3, label='SEW spectrum with young criterion')
plt.legend(fontsize=15)




a_cut=3700
b_cut=4150
plt.plot([a_cut,3750],[2,2.2],'k')
plt.plot([b_cut,6500],[2,2.2],'k')
plt.plot([a_cut,b_cut,b_cut,a_cut,a_cut],[0.5,0.5,2,2,0.5],'k')
plt.xlim(3500,9000)
plt.ylim(0.3,4.78)
plt.xlabel(r'$\lambda(\AA)$')
plt.ylabel('normed flux')

ax=plt.axes([0.111,0.737,0.45,0.2])
ax.patch.set_alpha(0)
plt.plot(p1.lam,p1.galaxy,'grey',lw=5, label = 'input spectrum')
plt.plot(lam_mock,galaxy0_mock,'k',lw=5, label = 'mock intrinsic spectrum')
plt.plot(p1.lam,p1.gal_temp@p1.weights,'dodgerblue',ls='--',lw=3, dashes=(10,5),label='SEW spectrum')
plt.plot(p1.lam,p1.gal_temp@p1t.weights,'r',ls='-.',lw=3 ,label='SEW spectrum with young crit')
plt.xlim(a_cut,b_cut)
plt.ylim(0.5,2)
if resp == 'sti':
    # plt.arrow(5100, 1.5, 0, -0.3, head_width=10, head_length=0.1, fc='k', ec='k')
    plt.annotate('stitching point', xy=(5100, 1.1), xytext=(5100, 1.5), arrowprops=dict(facecolor='black', shrink=0.05))

plt.axes([0.07,0.08,0.9,0.37])
plt.title('attenuation curve')
# plt.plot(p1.lam, p1.att_curve,'b.',label='SEW',alpha=0.3)

ebv=0.3
ext = lambda x,delta:(x/5500)**-delta
att = lambda x, ebv, delta: ebv*(ext(x,delta)-1)/(((89/110)**-delta)-1)

print(resp)
if resp == 'sti':
    resp_fun = lambda lam: -0.05*(np.sign(lam-5100)-1)
elif resp == 'cal':
    # resp = lambda lam: 0.1*(1-np.exp(-((lam-5500)/1500)**2))
    resp_fun = lambda lam: 0.05*np.sin(2*np.pi*(lam-5500)/2000)
elif resp is None:
    resp_fun = lambda lam: 0*lam


# plt.plot(p1.lam,att(p1.lam,ebv,1.3),'k',lw=5,label='mock')
plt.plot(p1.lam,att(p1.lam,ebv,1.3)+att(p1.lam,k*ebv,1.3),'k:',lw=5,label='mock young curve')
plt.plot(p1.lam,att(p1.lam,ebv,1.3),'k--',lw=5,label='mock old curve')


plt.plot(p1.lam,-2.5*np.log10((galaxy1-p1t.bestfit_emm)/(galaxy0-p1t.bestfit_emm)),'k',lw=5,label='mock tot curve')
# plt.plot(p1.lam,smooth(att(p1.lam,ebv,1.3)+2.5*(resp(p1.lam)),p1.lam,np.full_like(p1.lam,1),1e5,0.5)[0],'grey',ls='--',lw=5,label='mock curve+bias')

# plt.plot(p1.lam, -2.5*np.log10(p1.galaxy/(p1.gal_temp@p1.weights)),'b.',label='SEW',alpha=0.3,zorder=0)

plt.plot(p1.lam, p1.att_curve_smooth,'dodgerblue',ls='--',lw=3,label='SEW, tot curve')
plt.plot(p1.lam, p1t.att_curve_smooth,'r',ls=':',lw=3,label='SEW with young crit, old curve')
# plt.plot(p1.lam, -2.5*np.log10(p1.galaxy/(p1.gal_temp@p1t.weights)),'r.',alpha=0.3,zorder=0)
plt.plot(p1.lam, -2.5*np.log10(p1.galaxy_l/(p1.gal_temp_l@p1t.weights)),'r',label='SEW with young crit, tot curve',zorder=5)

# plt.plot(p1t.lam, p1t.att_curve_smooth,'dodgerblue',ls='--',lw=3,label='smoothed SEW')
# plt.plot(lam,2.5*np.log10((galaxy0-galaxy_emm)/(galaxy1-galaxy_emm)))

plt.xlabel(r'$\lambda(\AA)$')
plt.ylabel(r'$A_\lambda-A_V$')
plt.xlim(3550,9000)
# plt.ylim(-0.7,1.)
plt.legend(ncol=2,fontsize=15)

plt.tight_layout()

Appendix A

In [ ]:
"""
initial test environment
"""
from ppxf_sew import all_temp_make
from scipy.optimize import curve_fit,leastsq
from tqdm import trange
from ppxf.ppxf import ppxf as pp


def test_ewfit(j,ebv,delta,sn,v_delta=1e-4,l=1e-8,p=0.5,AGN=0,resp=None):
    n_star=120
    lam_range=np.array([3300,9200])
    
    wave = np.array([3798.983, 3836.479, 3890.158, 3971.202, 4102.899, 4341.691, 4862.691, 6564.632])  # vacuum wavelengths
    wave1=np.array([3726.032, 3728.815, 6716.44 , 6730.816, 3967.467, 3868.763,4685.704, 5875.614, 5006.843, 6300.298, 6583.452])
    ratios = np.array([ 0.0530, 0.0731, 0.105, 0.159, 0.259, 0.468, 1, 2.86]) # [0.0530, 0.0731, 0.105, 0.159,]
    ratios *= wave[-2]/wave  # Account for varying log-sampled pixel size in Angstrom
    num0=np.where((wave>lam_range[0])&(wave<lam_range[1]))
    num1=np.where((wave1>lam_range[0])&(wave1<lam_range[1]))
    weights=np.zeros(n_star+num0[0].shape[0]+num1[0].shape[0])
    num=np.arange(0,n_star)
    np.random.seed(j)
    np.random.shuffle(num)
    np.random.seed()
    weights[num[:10]]=1./10
    comp = np.array([0]*n_star + [1]*8 + [2]*11)
    gas = np.array(comp) >0
    params =np.array([50, 80, -10, 100, 30, 50])
    k=0
    temp_file=os.path.join(os.getcwd(),'sps_models/BC03_SSP_tpl_chab.fits')
    lam,galaxy1,galaxy0,galaxy1_sn, galaxy_emm,lam_temp0,temp0,velscale,noise=mock_file(temp_file,lam_range,weights,comp,params,gas,ebv,k,sn,v_delta=v_delta,delta=delta,AGN=AGN,resp=resp)
    c = 299792.458


    galaxy=galaxy1_sn

    noise=noise
    Z=0

    s=galaxy.shape[0]
    mask=np.zeros(s) #galaxy mask



    template=lib.temp(temp_file,velscale=velscale,FWHM_gal=2.76,FWHM_tem=2.5,normalize=True) #stellar template

    temp=template.templates.reshape(template.templates.shape[0],-1)
    lam_temp=np.exp(template.log_lam_temp)
    lam_range_gal = np.array([np.min(lam), np.max(lam)])

    templates,start,moments,component,gas_component,gas_names=all_temp_make(temp,lam_temp,lam_range_gal, tie_balmer=False, limit_doublets=False)
    # print(start)
    start = np.array([[50, 80], [-10, 100], [30, 50]])
    vlim = np.array([-2900, 2900])  # As 2e3 nonlinear_fit() +900 for 3sigma
    lam_range = lam_temp[[0, -1]]/np.exp(vlim/c)
    num=(lam > lam_range[0]) & (lam < lam_range[1])
    lam=lam[num]
    galaxy=galaxy[num]
    galaxy0=galaxy0[num]
    galaxy1=galaxy1[num]
    noise=noise[num]
    mask=mask[num]


    lin_method='cvxopt'#'lsq_box' #
    try:
        p1 = ewfit(templates, galaxy, noise, velscale, start,Z=Z, moments=moments ,dust=None,linear=True, linear_method=lin_method,plot=False,clean=False, degree=-1,lam=lam, lam_temp=lam_temp, component=component,gas_component=gas_component, gas_names=gas_names, tie_balmer= tie_balmer, limit_doublets=limit_doublets,l=l,p=p,quiet=True)
    except:
        p1 = ewfit(templates, galaxy, noise, velscale, start,Z=Z, moments=moments ,dust=None,linear=True,linear_method='lsq_box',plot=False,clean=False, degree=-1,lam=lam, lam_temp=lam_temp, component=component,gas_component=gas_component, gas_names=gas_names, tie_balmer= tie_balmer, limit_doublets=limit_doublets,l=l,p=p,quiet=True)

    num=np.where(p1.att_curve==p1.att_curve)
    ext = lambda x,delta:(x/5500)**-delta
    att = lambda x, ebv, delta: ebv*(ext(x,delta)-1)/(((89/110)**-delta)-1)
    p1att=curve_fit(att,p1.lam[num], p1.att_curve[num],p0=[ebv,delta],bounds=((-0.1,0.5),(1,2)))[0]
    lamt=np.linspace(3500,9000,1000)
    f=interp1d(p1.lam,p1.att_curve_smooth,kind='cubic')
    att_curve_smooth=f(lamt)

    return att_curve_smooth,p1att
    # return np.array([age,metal]),p1,weights

In [ ]:
from tqdm import trange
from multiprocessing import cpu_count
np.random.seed(0)
size=1000
ls=np.random.randint(1000000,size=size)


l_list=np.array([1e5,1e6,1e7,1e8])
ebv_list=[0,0.1,0.3,0.5]
delta=1.3
sn_list=[5,15,30]
lamt=np.linspace(3500,9000,1000)
att_curve=np.zeros((len(ebv_list),len(sn_list),size,1000))
para=np.zeros((len(ebv_list),len(sn_list),size,2))


In [ ]:

pool=Pool(processes=min(cpu_count(),64))

for r in range(4):

    for l in range(3):
        fun0=partial(test_ewfit,ebv=ebv_list[r],v_delta=1e-4,delta=delta,sn=sn_list[l],l=1e-8,p=0.5,AGN=0.,resp=None)
        res0= [pool.apply_async(fun0,args=(ls[k],)) for k in range(size)]
        for k in trange(size): 
            att_curve[r,l,k],para[r,l,k] = res0[k].get() 

pool.close()
pool.join()

In [ ]:
lam0=np.array([lamt for i in range(1000)])
from matplotlib import colors
np.random.seed(114)
size=100
ls=np.random.randint(100000,size=size)
ebv_list=[0.,0.1,0.3,0.5]
delta_list=[0.7,1,1.3,1.5]
sn_list=[5,15,30]
color=['r','g','b']
plt.figure(figsize=(20,15))
ext = lambda x,delta:(x/5500)**-delta
att = lambda x, ebv, delta : ebv*(ext(x,delta)-1)/(((89/110)**-delta)-1)

# data=curve_fit(att,p1.lam[num], p1.att_curve[num],p0=[ebv,delta,0],bounds=((-0.1,0.5,-0.5),(0.5,2,0.5)))[0]
rand=np.random.randint(size,size=100)
j=2
for i in range(4):
    for q in range(3):
        ax=plt.subplot(4,4,4*i+q+1)
      
        ebv=ebv_list[i]
        delta=delta_list[2]
        sn=sn_list[q]
        ebv0=para[i,q,:,0] 
        delta0=para[i,q,:,1]
        curve=att_curve[i,q]

        num=np.where(delta0!=0)
        numt=np.where(curve==curve)

        ax.plot(lam,att(lam,ebv,delta),'k')
        ax.plot(lam,np.nanmedian(curve[num],0),c='dodgerblue',ls='--')


        ax.set_xlim(3500,9000)
        ax.set_ylim(-1.2,1.5)
        xax=ax.get_xlim()
        yax=ax.get_ylim()
        ax.hist2d(lam0[numt],curve[numt],bins=100,range=[xax,yax],norm = colors.LogNorm(),cmap ="PuBu")

        x_pos = xax[1] - 0.47 * (xax[1] - xax[0])
        y_pos = yax[1] - 0.1 * (yax[1] - yax[0])
        ax.text(xax[1] - 0.6 * (xax[1] - xax[0]), yax[1] - 0.2 * (yax[1] - yax[0]), 
                r'$\overline{\rm{E(B-V)}}=%s,\bar{\beta}=%s$'%(round(np.median(ebv0[num]),3),round(np.median(delta0[num]),3)),
                wrap=True,ha='left',fontsize=13)
        ax.text(xax[1] - 0.6 * (xax[1] - xax[0]), yax[1] - 0.3 * (yax[1] - yax[0]), 
                r'$\sigma_{\rm{E(B-V)}}=%s,\sigma_{\beta}=%s$'%(round(np.std(ebv0[num]),3),round(np.std(delta0[num]),3)),
                wrap=True,ha='left',fontsize=13)


        ax.text(xax[1] - 0.8 * (xax[1] - xax[0]), y_pos, r'$\rm{S/N}=%s,\rm{E(B-V)}=%s,\beta=%s$'%(sn,ebv,delta),fontsize=15)
        if i==3:         
            ax.set_xlabel(r'$wavelegth(\AA)$')
        if q==0:
            ax.set_ylabel(r'$A(\lambda)-A_V$')

        ax=plt.subplot(4,4,4*i+4)
        num=np.where(delta0!=0)
        sns.kdeplot(x=ebv0[num],y=delta0[num],levels=4,color=color[q],alpha=0.7,linestyles='--')
        ax.plot(np.mean(ebv0[num]),np.mean(delta0[num]),'X',c=color[q],alpha=0.7,markersize='10')
        if q==0:
            xax_kde=ax.get_xlim()
            yax_kde=ax.get_ylim()
        # ax.plot([-10,-9],[0,1],color=color[q],alpha=0.7,ls='--',label='S/N=%s'%(sn))
        ax.set_xlim(xax_kde)
        ax.set_ylim(yax_kde)
        if i==3:
            ax.set_xlabel('E(B-V)')
        ax.set_ylabel(r'$\beta$')

        y_pos = yax_kde[1] - 0.1 * (yax_kde[1] - yax_kde[0])
        ax.text(xax_kde[1] - 0.9 * (xax_kde[1] - xax_kde[0]), y_pos, r'$\rm{E(B-V)}=%s,\beta=%s$'%(ebv,delta),fontsize=15)

    
plt.tight_layout()
